In [ ]:
import os
import random
import math
import time

In [ ]:
class gate_dlinkedList:
    def __init__(self, gate_id, bucket_side, area):
        self.gate_id = gate_id
        self.bucket_side = bucket_side
        self.area = area
        self.lock = False
        self.gain = None
        self.next = None
        self.prev = None

    def set_next(self, next_gate):
        self.next = next_gate
        if next_gate:
            next_gate.prev = self
        return self.next
    
    def move_gate(self, new_gain):
        self.remove_gate()
        self.gain = new_gain
        if new_gain in self.bucket_side.keys():
            self.bucket_side[new_gain] = self.bucket_side[new_gain].set_next(self)
            self.bucket_side[new_gain].next = None
        else:
            self.bucket_side[new_gain] = self
            self.bucket_side[new_gain].next = None
            self.bucket_side[new_gain].prev = None
            
    def remove_gate(self):
        if self.gain in self.bucket_side.keys() and not self.next:
            if self.prev:
                self.bucket_side[self.gain] = self.prev
            else: #not self.prev
                del self.bucket_side[self.gain]
        if self.prev:
            self.prev.next = self.next
        if self.next:
            self.next.prev = self.prev
        self.next = None
        self.prev = None
        self.gain = None

    def __repr__(self):
        next_id = self.next.gate_id if self.next else 'None'
        prev_id = self.prev.gate_id if self.prev else 'None'
        return f"Gate(Id:{self.gate_id},next:{next_id},prev:{prev_id},lock:{self.lock},gain:{self.gain})"

In [ ]:
class readFile:
    def __init__(self):
        self.circuit_data = None
        
    def read_circuit(self, circuit_folder_path):
        netD_file = None
        are_file = None
        for file in os.listdir(circuit_folder_path):
            if file.endswith('.netD'):
                netD_file = os.path.join(circuit_folder_path, file)
            if file.endswith('.are'):
                are_file = os.path.join(circuit_folder_path, file)
        if netD_file and are_file:
            self.circuit_data = self.parse_net_files(netD_file, are_file)
            return self.circuit_data
        else:
            print("Error Message: Missing Files")
            return None

    def parse_net_files(self, netD_file, are_file):
        # network information
        nets_out, nets_in, connections, nodes, modules, pad_offset = self.read_netD_file(netD_file)
        # module areas
        areas = self.read_are_file(are_file)
        # network information with pin directions
        self.circuit_data = {
            'areas': areas,
            'netD_out': nets_out,
            'netD_in': nets_in,
            'connections': int(connections),
            'nodes': int(nodes),
            'modules': int(modules),
            'pad_offset': int(pad_offset)
        }
        return self.circuit_data
    
    def read_are_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        module_areas = {}
        for line in lines:
            module_id, area = line.split()
            module_areas[module_id] = int(area)
        return module_areas

    def read_netD_file(self, circuit_folder_path):
        with open(circuit_folder_path, 'r') as file:
            lines = file.readlines()
        self.circuit_info = tuple(item.replace('\n', '') for item in lines[0:5])
        _, connections, nodes, modules, pad_offset = self.circuit_info
        lines = lines[5:]
        nets_out = {}
        nets_in = {}
        current_key = None
        for line in lines:
            tokens = line.split()
            #populate dictionaries
            if tokens[0] not in nets_out.keys():
                nets_out[tokens[0]] = []
            if tokens[0] not in nets_in.keys():
                nets_in[tokens[0]] = []
            if tokens[1] == 's':
                current_key = tokens[0]
            elif tokens[1] == 'l':
                #nets out
                nets_out[current_key].append(tokens[0])
                #nets in
                nets_in[tokens[0]].append(current_key)
        return nets_out, nets_in, connections, nodes, modules, pad_offset

In [ ]:
class FM_algo:
    def __init__(self, circuit_folder_path, max_iteration, speedrun, ratio = 0.5, tolerance = 0.1):
        self.start_time = time.time()
        self.circuit_data = readFile().read_circuit(circuit_folder_path)
        self.circuit_connections_in = self.circuit_data['netD_in']
        self.circuit_connections_out = self.circuit_data['netD_out']
        self.circuit_gate_areas = self.circuit_data['areas']
        self.ratio, self.tolerance = ratio, tolerance
        self.W = sum(self.circuit_gate_areas.values())
        self.s_max = max(self.circuit_gate_areas.values())
        self.k = self.tolerance * len(self.circuit_gate_areas) 
        self.bucket_count1, self.bucket_count2 = 0, 0
        self.bucket_gains1 = dict()
        self.bucket_gains2 = dict()
        self.vertex = dict()
        self.max_iteration = max_iteration
        self.speedrun = speedrun
        initial_cutsize, best_cutsize, improvement, iter_count = self.fd_algo()
        runtime = time.time() - self.start_time
        # area1, area2 = self.get_partition_areasums(self.bucket_gains1), self.get_partition_areasums(self.bucket_gains2)
        area1, area2 = sum(self.partition1.values()), sum(self.partition2.values())
        if self.speedrun == False:
            print('input connections', self.circuit_connections_in)
            print('output connections', self.circuit_connections_out)

        print(f'Initial Cut Size: {initial_cutsize}')
        print('Fiduccia-Mattheyses Algorithm Complete!')
        print(f'Runtime: {runtime} seconds')
        print(f'Best Cut Size: {best_cutsize}')
        print(f'Total number of iterations: {iter_count}')
        print(f'Improvement percentage: {improvement}%')
        print(f'Number of Gates in Bucket 1: {self.bucket_count1}')
        print(f'Number of Gates in Bucket 2: {self.bucket_count2}')
        print(f'Final Area Partitions1: {area1}')
        print(f'Final Area Partitions2: {area2}')
        print(f'Final Partitions1: {self.partition1}')
        print(f'Final Partitions2: {self.partition2}')
    def fd_algo(self):
        #partition circuit
        self.partition1, self.partition2 = self.optimal_partition_circuit()
        print(f'Initial Area Partition 1: {sum(self.partition1.values())}')
        print(f'Initial Area Partition 2: {sum(self.partition2.values())}')
        if self.speedrun == False:
            print('parition1 ',self.partition1)
            print('parition2 ',self.partition2)
        #initialize buckets
        self.bucket_gains1 = self.initialize_buckets(self.partition1, self.bucket_gains1)
        self.bucket_gains2 = self.initialize_buckets(self.partition2, self.bucket_gains2)
        cutsize = self.calculate_cutsize()
        initial_cutsize = cutsize
        ##TODO: Begin Fiduccia-Mattheyses Algorithm below
        max_iter_count = self.max_iteration
        iter_count = 0
        best_cutsize = float('inf') # Initialize the best cut size to inf
        
        while(True):
            iter_count += 1
            print(f"Number of Iterations: {iter_count}")
            last_cutsize = cutsize
            last_bucket1 = self.bucket_gains1
            last_bucket2 = self.bucket_gains2
            cutsize = self.fm_pass()
            if (cutsize < best_cutsize) and (math.isfinite(cutsize)): #if improvements
                best_cutsize = cutsize           
            if (cutsize >= last_cutsize) and (iter_count >= max_iter_count) or (last_cutsize == 1): #if no improvement
                #roll back to initial best buckets
                self.last_bucket1 = last_bucket1
                self.last_bucket2 = last_bucket2
                break   
            self.reinitialize_buckets()
        improvement = round((1-best_cutsize/initial_cutsize)*100, 4) if math.isfinite(best_cutsize) else 0          
        return initial_cutsize, best_cutsize, improvement, iter_count
    
    def fm_pass(self):
        #initialize best cutsize
        best_cutsize = float('inf')
        best_bucket1 = self.bucket_gains1
        best_bucket2 = self.bucket_gains2
        #Reset parameters for optimization
        free_gates = self.circuit_gate_areas.copy()
        self.W = sum(self.circuit_gate_areas.values())
        self.s_max = max(self.circuit_gate_areas.values())
        self.k = self.tolerance * len(self.circuit_gate_areas) 
        locked_gates_count = 0
        print('Initial Partition1: ', self.partition1)
        print('Initial Partition2: ', self.partition2)

        while (not self.check_gate_locks(locked_gates_count)): #Stop if all gates are locked
            optimal_moves = self.get_optimal_moves(free_gates)
            if not optimal_moves:
                break
            optimal_max_gain_node = self.calculate_optimal_maxgain(optimal_moves)
            #move to opposite side of bucket && remove gate from bucket & lock gate
            optimal_max_gain_node.remove_gate()
            optimal_max_gain_node.lock = True
            locked_gates_count += 1
            if self.speedrun == False:
                print(f'FM progress: {locked_gates_count} / {len(self.vertex)}')
            if optimal_max_gain_node.bucket_side == self.bucket_gains1:
                optimal_max_gain_node.bucket_side = self.bucket_gains2
                self.bucket_count1 -= 1
                self.bucket_count2 += 1
                del self.partition1[optimal_max_gain_node.gate_id]
                self.partition2[optimal_max_gain_node.gate_id] = optimal_max_gain_node.area
            elif optimal_max_gain_node.bucket_side == self.bucket_gains2:
                optimal_max_gain_node.bucket_side = self.bucket_gains1
                self.bucket_count1 += 1
                self.bucket_count2 -= 1
                del self.partition2[optimal_max_gain_node.gate_id]
                self.partition1[optimal_max_gain_node.gate_id] = optimal_max_gain_node.area
            # Update gain locally for affected gates
            self.update_local_gains(optimal_max_gain_node)
            #TODO: Update W, s_max, and k
            self.update_optimization_parameters(free_gates, optimal_max_gain_node.gate_id, locked_gates_count)
            current_cutsize = self.calculate_cutsize()
            #Rollback to best observed cutsize
            if 1 <= current_cutsize < best_cutsize:
                best_cutsize = current_cutsize
                best_bucket1 = self.bucket_gains1
                best_bucket2 = self.bucket_gains2
                best_partition1 = self.partition1
                best_partition2 = self.partition2
            
        self.bucket_gains1 = best_bucket1
        self.bucket_gains2 = best_bucket2
        self.partition1 = best_partition1
        self.partition2 = best_partition2
        new_cutsize = best_cutsize
        return new_cutsize
    
    def update_local_gains(self, moved_gate):
        self.update_local_gains_for_connections(moved_gate, self.circuit_connections_out)
        self.update_local_gains_for_connections(moved_gate, self.circuit_connections_in)

    def update_local_gains_for_connections(self, moved_gate, connections):
        for net in connections[moved_gate.gate_id]:
            net_node = self.vertex[net]
            net_node.move_gate(self.calculate_gain(net))
    
    def update_optimization_parameters(self, gate_areas, removed_gate, locks):
        del gate_areas[removed_gate]
        # Update W, s_max, and k
        self.W = sum(gate_areas.values())
        self.s_max = max(gate_areas.values())
        num_free_cells = len(self.vertex) - locks
        self.k = self.tolerance * num_free_cells
        
    def calculate_optimal_maxgain(self, optimal_moves):
        if not optimal_moves: return None
        max_gain = self.vertex[optimal_moves[0][0]].gain
        optimal_balance = float('inf')
        partition_sum1 = sum(self.partition1.values())
        partition_sum2 = sum(self.partition1.values())
        for id, _ in optimal_moves:
            if self.vertex[id].gain != max_gain:
                break
            else:
                node = self.vertex[id]
                if node in self.bucket_gains1:
                    new_size1 = (partition_sum1) - node.area
                    new_size2 = (partition_sum2) + node.area
                else: # if node in self.bucket_gains2
                    new_size2 = (partition_sum2) - node.area
                    new_size1 = (partition_sum1) + node.area
                delta = abs((new_size1 - new_size2) - self.ratio * self.W)
                if delta < optimal_balance:
                    optimal_balance = self.vertex[id]
        return optimal_balance


    def reinitialize_buckets(self):
        for node in self.vertex.values():
            #unlock gate
            node.lock = False
            gain = self.calculate_gain(node.gate_id)
            node.move_gate(gain)

    def check_gate_locks(self, locked_gates_count):
        # return True if all gates are locked
        return locked_gates_count == len(self.vertex) 

    def calculate_cutsize(self):
        cut_size = 0
        for gate, connected_gates in self.circuit_connections_out.items():
            current_gate_node = self.vertex[gate]
            current_gate_side = current_gate_node.bucket_side
            for connected_gate in connected_gates:
                connected_gate_node = self.vertex[connected_gate]
                connected_gate_side = connected_gate_node.bucket_side
                if current_gate_side != connected_gate_side:
                    cut_size += 1
                    break 
        return cut_size

    def calculate_max_gain(self):
        max_gain1 = max(self.bucket_gains1.keys(), default=float('-inf'))
        max_gain2 = max(self.bucket_gains2.keys(), default=float('-inf'))
        if max_gain1 > max_gain2:
            selected_bucket, _ = self.bucket_gains1, self.bucket_gains2
        elif max_gain1 < max_gain2:
            selected_bucket, _ = self.bucket_gains2, self.bucket_gains1
        else:
            selected_bucket, _ = (
                self.bucket_gains1, self.bucket_gains2) if self.bucket_count1 > self.bucket_count2 else (
                self.bucket_gains2, self.bucket_gains1)
        overall_max_gain = max(max_gain1, max_gain2)
        return selected_bucket.get(overall_max_gain), overall_max_gain
    
    def optimal_partition_circuit(self):
        partition1, partition2 = {}, {}
        target_area = self.ratio * self.W
        sorted_gates = sorted(self.circuit_gate_areas.items(), key=lambda x: x[1], reverse = True) 
        for gate, area in sorted_gates:
            if sum(partition1.values()) + area <= target_area:
                partition1[gate] = area
            else:
                partition2[gate] = area
        return partition1, partition2

    def get_optimal_moves(self, circuit_area):
        optimal_moves = []
        # area_sums1 = self.get_partition_areasums(self.bucket_gains1)
        # area_sums2 = self.get_partition_areasums(self.bucket_gains2)
        area_sums1 = sum(self.partition1.values())
        area_sums2 = sum(self.partition2.values())
        for gate in circuit_area.keys():
            node = self.vertex[gate]
            if node.bucket_side == self.bucket_gains1:
                source_partiton_area = area_sums1 - node.area
                dest_partition_area = area_sums2 + node.area
            else: # if node.bucket_side == self.bucket_gains2
                source_partiton_area = area_sums2 - node.area
                dest_partition_area = area_sums1 + node.area
            #BALANCE CRITERION --> rW - k * s_max <= |Partition area| <= rW + k * s_max
            source_criterion = (self.ratio * self.W) - (self.k * self.s_max) <= source_partiton_area <= (self.ratio * self.W) + (self.k * self.s_max)
            dest_criterion = (self.ratio * self.W) - (self.k * self.s_max) <= dest_partition_area <= (self.ratio * self.W) + (self.k * self.s_max)
            if (source_criterion or dest_criterion) and not node.lock:
                optimal_moves.append((gate, node.gain))
        optimal_moves.sort(key=lambda x: x[1], reverse=True)
        return optimal_moves

    def partition_circuit(self):
        circuit_nodes = list(self.circuit_connections_out.keys())
        random.shuffle(circuit_nodes)
        partition = len(circuit_nodes) // 2
        partition1 = circuit_nodes[:partition]
        partition2 = circuit_nodes[partition:] 
        return partition1, partition2
    
    def calculate_initial_gain(self, gate):
        external_sum = 0
        internal_sum = 0
        current_partition = self.partition1 if gate in self.partition1 else self.partition2
        #output connections
        for next_gate in self.circuit_connections_out[gate]:
            internal_sum += 1 if next_gate in current_partition else 0
            external_sum += 1 if next_gate not in current_partition else 0
        #input connnections
        for prev_gate in self.circuit_connections_in[gate]:
            internal_sum += 1 if prev_gate in current_partition else 0
            external_sum += 1 if prev_gate not in current_partition else 0
        #calculate gain
        return external_sum - internal_sum

    def calculate_gain(self, gate):
        external_sum = 0
        internal_sum = 0
        current_partition = self.vertex[gate].bucket_side
        #output connections
        for next_gate in self.circuit_connections_out[gate]:
            next_gate_side = self.vertex[next_gate].bucket_side
            internal_sum += 1 if next_gate_side == current_partition else 0
            external_sum += 1 if next_gate_side != current_partition else 0
        #input connnections
        for prev_gate in self.circuit_connections_in[gate]:
            prev_gate_side = self.vertex[prev_gate].bucket_side
            internal_sum += 1 if prev_gate_side == current_partition else 0
            external_sum += 1 if prev_gate_side != current_partition else 0   
        #calculate gain
        return external_sum - internal_sum
        
    def initialize_buckets(self, partition, bucket):
        for gate, area in partition.items():
            current_gate = gate_dlinkedList(gate, bucket, area)
            self.vertex[gate] = current_gate
            gain = self.calculate_initial_gain(gate)
            current_gate.gain = gain
            if gain in bucket.keys():
                bucket[gain] = bucket[gain].set_next(current_gate)
                bucket[gain].next = None
            else:
                bucket[gain] = current_gate #gate must be doubly linked list OOC
                bucket[gain].prev = None
                bucket[gain].next = None
            if bucket == self.bucket_gains1:
                self.bucket_count1 += 1
            else:
                self.bucket_count2 += 1
        return bucket 

In [ ]:
def main():
    circuit_folder_path = "test_simple"
    fm = FM_algo(circuit_folder_path, 5, speedrun=True)
if __name__ == "__main__":
    main()

In [ ]:
def main():
    circuit_folder_path = "test3"
    fm = FM_algo(circuit_folder_path, 1, speedrun=True)
if __name__ == "__main__":
    main()

In [ ]:
def main():
    circuit_folder_path = "test5"
    fm = FM_algo(circuit_folder_path, 1, speedrun=True)
if __name__ == "__main__":
    main()